In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
colors = {
    "DQN": 'darkorange',
    "PPO": 'forestgreen',
    "CMA": 'cyan',
    "ODT": 'blueviolet'
}

In [ ]:
starting_exp = 4000
ending_exp = 4035
save_processed_data = True
#exp_lists_preformatted = ['4000_4035.csv', '4072_4107.csv'] # Existing formatted datasets
exp_lists_preformatted = []


experiments = range(starting_exp, ending_exp + 1)

exp_agent_data = []

agg_counts = []

# REMOVE THIS
#algos = ['DQN', 'PPO', 'CMA', 'ODT']

if len(exp_lists_preformatted) == 0:
    for ind, exp_num in enumerate(experiments):
        config_fname = f'./Exp_{exp_num}/config.yaml'
        
        c = load_config_file(config_fname)
        nn_c = c['nn_hyperparameters']
        federated_c = c['federated_learning_settings']
        algo_c = c['algorithm_settings']
        env_c = c['environment_settings']
        eval_c = c['eval_config']
    
        ev_info = []
    
        seed = env_c['seed']
    
        algorithm_dm = algo_c['algorithm']
        
        def load_from_json_with_error_handling(filepath, columns_specific):
            try:
                return read_csv_data(filepath, columns=columns_specific)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
                return None  # Handle the error and return None or an empty object
        
        
        d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
        
        if not os.path.exists(d_base):
            d_base = f"../metrics/Exp_{exp_num}"
                
        base_path = f"{d_base}/train/metrics"
    
        print(f'Loading {base_path}_agent_metrics.csv')
        agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'agent_index', 'reward',])
        
        agent_data['seed'] = seed
        agent_data['exp_num'] = exp_num
        
        agent_data['algorithm'] = algorithm_dm
        # agent_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
        
        agent_data['season'] = env_c['season']
        agent_data['num_aggs'] = federated_c['aggregation_count']
        agent_data['eps_per_agg'] = nn_c['num_episodes']
    
        exp_agent_data.append(agent_data)

Loading ../../../../storage_1/metrics/Exp_4000/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4001/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4002/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4003/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4004/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4005/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4006/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4007/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4008/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4009/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4010/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4011/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_4012/train

In [ ]:
if len(exp_lists_preformatted) == 0:
    # Convert data to DataFrame for easier manipulation
    df_agent = pd.concat(exp_agent_data, ignore_index=True)
    
    cumulative_agent_df = (
        df_agent
        .groupby(
            ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season', 'num_aggs', 'eps_per_agg']
        )['reward']
        .sum()
        .reset_index()
    )
    
    # Rename the 'reward' column to 'cumulative_reward' for clarity
    cumulative_agent_df.rename(columns={'reward': 'cumulative_reward'}, inplace=True)
    
    # Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
    cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * cumulative_agent_df['eps_per_agg'] + cumulative_agent_df['episode']
    
    cumulative_agent_df.head()

In [ ]:
# Define the file path with starting_exp and ending_exp variables
file_path_for_processed_data = f'../../../../storage_1/metrics/formatted_experiment_data/part_1/'

if len(exp_lists_preformatted) == 0:
    cumulative_avg_reward_by_algorithm = (
        cumulative_agent_df
        .groupby(['episode', 'algorithm', 'seed', 'num_aggs', 'season'])['cumulative_reward']
        .mean()
        .reset_index()
    )
    
    cumulative_avg_reward_by_algorithm = cumulative_avg_reward_by_algorithm.sort_values(
        ['season', 'algorithm', 'seed', 'num_aggs', 'episode']
    )
    
    cumulative_avg_reward_by_algorithm['cumulative_reward'] = (
        cumulative_avg_reward_by_algorithm
        .groupby(['algorithm', 'seed', 'num_aggs', 'season'])['cumulative_reward']
        .transform(lambda x: x.expanding().mean())
    )

    if save_processed_data:
        if not os.path.exists(file_path_for_processed_data):
            os.makedirs(file_path_for_processed_data)
        
        # Save the DataFrame to a CSV file
        cumulative_avg_reward_by_algorithm.to_csv(f"{file_path_for_processed_data}/{starting_exp}_{ending_exp}.csv", index=False)
        
        print(f"Data successfully saved to {file_path_for_processed_data}/{starting_exp}_{ending_exp}.csv")

if len(exp_lists_preformatted) > 0:
    dataframes = []
    
    # Load and combine datasets
    for existing_set in exp_lists_preformatted:
        loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/{existing_set}")
        # Combine datasets
        dataframes.append(loaded_dataset)

    cumulative_avg_reward_by_algorithm = pd.concat(dataframes, ignore_index=True)

In [ ]:
cumulative_avg_reward_by_algorithm['num_aggs'] = np.where(
    (cumulative_avg_reward_by_algorithm['num_aggs'] == 3) & 
    (cumulative_avg_reward_by_algorithm['algorithm'] == 'CMA'), 
    1, 
    cumulative_avg_reward_by_algorithm['num_aggs']
)

agg_levels_sorted = sorted(cumulative_avg_reward_by_algorithm['num_aggs'].unique(), reverse=False)

num_plots = len(cumulative_avg_reward_by_algorithm['num_aggs'].unique())

fig, axes = plt.subplots(1, num_plots, figsize=(24, 6), sharey=True)  # Adjust figsize as needed

for plot_ind, agg_level in enumerate(agg_levels_sorted):
    ax = axes[plot_ind]
    
    agg_level_data = cumulative_avg_reward_by_algorithm.loc[cumulative_avg_reward_by_algorithm['num_aggs'] == agg_level]

    total_eps = agg_level_data['episode'].max()
    eps_per_agg = np.ceil(total_eps / agg_level)

    for season in agg_level_data['season'].unique():
        season_data = agg_level_data[agg_level_data['season'] == season]
        
        for algo in season_data['algorithm'].unique():
            # Filter the data for the current zone
            algo_data = season_data[season_data['algorithm'] == algo]
            
            min_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].min()
            max_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].max()
            mean_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].mean()
        
            ax.fill_between(
                min_cumulative_avg_reward.index, 
                min_cumulative_avg_reward.values, 
                max_cumulative_avg_reward.values,
                color=colors[algo],
                alpha=0.3
            )
            ax.plot(
                mean_cumulative_avg_reward.index, 
                mean_cumulative_avg_reward.values,
                color=colors[algo],
                label=f'Algorithm {algo} Average Reward - Season {season}'
            )

    for agg in range(1, agg_level + 1):
        if agg == 1:
            ax.axvline(x=(agg * eps_per_agg), color='r', linestyle='--', linewidth=1, label='Aggregation', alpha=0.3)
        else:
            ax.axvline(x=(agg * eps_per_agg), color='r', linestyle='--', linewidth=1, alpha=0.3)
    
    ax.set_xlabel('Episode')
    axes[0].set_ylabel('Cumulative Average Reward')
    ax.set_title(f'{agg_level} Aggregations')
    ax.legend(loc='lower right')
    ax.grid(True)

fig.show()

In [ ]:
agg_levels_sorted = sorted(cumulative_avg_reward_by_algorithm['num_aggs'].unique(), reverse=True)

num_plots = len(cumulative_avg_reward_by_algorithm['num_aggs'].unique())

fig, axes = plt.subplots(num_plots, 1, figsize=(8, 16))  # Adjust figsize as needed

for plot_ind, agg_level in enumerate(agg_levels_sorted):
    ax = axes[plot_ind]
    
    agg_level_data = cumulative_avg_reward_by_algorithm.loc[cumulative_avg_reward_by_algorithm['num_aggs'] == agg_level]

    total_eps = agg_level_data['episode'].max()
    eps_per_agg = np.ceil(total_eps / agg_level)

    for season in agg_level_data['season'].unique():
        season_data = agg_level_data[agg_level_data['season'] == season]
        
        for algo in season_data['algorithm'].unique():
            # Filter the data for the current zone
            algo_data = season_data[season_data['algorithm'] == algo]
            
            min_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].min()
            max_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].max()
            mean_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].mean()
        
            ax.fill_between(
                min_cumulative_avg_reward.index, 
                min_cumulative_avg_reward.values, 
                max_cumulative_avg_reward.values, 
                alpha=0.3
            )
            ax.plot(
                mean_cumulative_avg_reward.index, 
                mean_cumulative_avg_reward.values, 
                label=f'Algorithm {algo} Average Reward - Season {season}'
            )

    for agg in range(1, agg_level + 1):
        if agg == 1:
            ax.axvline(x=(agg * eps_per_agg), color='r', linestyle='--', linewidth=1, label='Aggregation')
        else:
            ax.axvline(x=(agg * eps_per_agg), color='r', linestyle='--', linewidth=1)
        
    ax.set_xlabel('Episode')
    ax.set_ylabel('Cumulative Average Reward')
    ax.set_title(f'{agg_level} Aggregations')
    ax.legend(loc='lower right')
    ax.grid(True)

fig.show()